<a href="https://colab.research.google.com/github/lsteffenel/CHPS0704/blob/main/TP3/6_kmeans_cah_fromage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

K-means

On utilise un dataset "Fromage" disponible sur le site de Marie Chavent à Bordeaux.

In [ ]:
import pandas as pnd

fromage = pnd.read_csv('https://raw.githubusercontent.com/lsteffenel/CHPS0704/refs/heads/main/data/fromage.csv', delimiter=';',  header=[0],index_col=[0])
fromage.info()

In [ ]:
fromage.head()

In [ ]:
%matplotlib inline

In [ ]:
import seaborn as sns
grid = sns.pairplot(data=fromage, vars=fromage.columns[0:8])

## KMeans

Dans un premier temps nous allons utiliser kMeans pour trouver des "groupes" de fromage.

In [ ]:
from sklearn.cluster import KMeans
kmeansf = KMeans(n_clusters=4, random_state=11)
kmeansf.fit(fromage)

In [ ]:
print(kmeansf.labels_)

In [ ]:
import numpy as np
idk = np.argsort(kmeansf.labels_)

In [ ]:
print(pnd.DataFrame(fromage.index[idk],kmeansf.labels_[idk]))

In [ ]:
#librairie pour évaluation des partitions
from sklearn import metrics
import numpy as np

#utilisation de la métrique "silhouette"
#faire varier le nombre de clusters de 2 à 10
res = np.arange(9,dtype="double")
for k in np.arange(9):
    km = KMeans(n_clusters=k+2)
    km.fit(fromage)
    res[k] = metrics.silhouette_score(fromage,km.labels_)
print(res)

#graphique
import matplotlib.pyplot as plt
plt.title("Silhouette")
plt.xlabel("# of clusters")
plt.plot(np.arange(2,11,1),res)
plt.show()

## Question :
À votre avis (et en fonction du graphique Silhouette), quel serait un bon nombre de groupes de fromage ? Vous pouvez "tricher" et analyser les caractéristiques des fromages appartenant à chaque groupe.

# Clustering Hierarchique

Maintenant nous allons utiliser une autre méthode de clustering, la CAH (Clustering Agglomératif Hierarchique).
L'objectif ici est moins de déterminer le nombre optimal de cluster mais de visualiser les groupes.

In [ ]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [ ]:
Z = linkage(fromage,method='ward',metric='euclidean')

In [ ]:
plt.title("CAH")
dendrogram(Z,labels=fromage.index,orientation='left',color_threshold=300)
plt.show()

Disons qu'on veut s'arrêter à 4 clusters. On fera cela en "coupant" les clustering à l'hauteur t=300

In [ ]:
groupes_cah = fcluster(Z,t=300,criterion='distance')
print(groupes_cah)

In [ ]:
import numpy as np
idg = np.argsort(groupes_cah)

In [ ]:
print(pnd.DataFrame(fromage.index[idg],groupes_cah[idg]))

Enfin, on compare les clusters obtenus avec kMeans et ceux de CAH. Est-ce qu'ils sont similaires ou divergent beaucoup ? (Attention : comme les labels de kMeans et de CAH sont crées de manière aléatoire, il se peut que les indexes ne soient pas identiques)

In [ ]:
pnd.crosstab(groupes_cah,kmeansf.labels_)